<h1> Movie review classification with NLTK </h1>

In [16]:
#modificado en vs sutidio
#aca hay otro cambio
import random
import nltk
from nltk import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
cats = movie_reviews.categories()
reviews = []
for cat in cats:
    for fid in movie_reviews.fileids(cat):
        review = (list(movie_reviews.words(fid)),cat)
        reviews.append(review)
random.shuffle(reviews)

In [3]:
all_wd_in_reviews = nltk.FreqDist(wd.lower() for wd in movie_reviews.words())
top_wd_in_reviews = [list(wds) for wds in zip(*all_wd_in_reviews.most_common(2000))][0]

In [4]:
def ext_ft(review,top_words):
    review_wds = set(review)
    ft = {}
    for wd in top_words:
        ft['word_present({})'.format(wd)] = (wd in review_wds)
    return ft

In [5]:
featuresets = [(ext_ft(d,top_wd_in_reviews), c) for (d,c) in reviews]
train_set, test_set = featuresets[200:], featuresets[:200]

In [6]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.86


In [7]:
classifier.show_most_informative_features(20)

Most Informative Features
word_present(outstanding) = True              pos : neg    =     12.3 : 1.0
     word_present(mulan) = True              pos : neg    =      8.8 : 1.0
    word_present(seagal) = True              neg : pos    =      8.0 : 1.0
word_present(wonderfully) = True              pos : neg    =      6.5 : 1.0
     word_present(damon) = True              pos : neg    =      5.8 : 1.0
      word_present(lame) = True              neg : pos    =      5.4 : 1.0
     word_present(waste) = True              neg : pos    =      5.4 : 1.0
       word_present(era) = True              pos : neg    =      5.3 : 1.0
    word_present(wasted) = True              neg : pos    =      5.1 : 1.0
     word_present(awful) = True              neg : pos    =      5.1 : 1.0
    word_present(poorly) = True              neg : pos    =      4.9 : 1.0
word_present(ridiculous) = True              neg : pos    =      4.8 : 1.0
 word_present(laughable) = True              neg : pos    =      4.4 : 1

In [8]:
dict_vectorizer=None
def get_train_test(train_set,test_set):
    global dict_vectorizer
    dict_vectorizer = DictVectorizer(sparse=False)
    X_train, y_train = zip(*train_set)
    X_train = dict_vectorizer.fit_transform(X_train)
    X_test,y_test = zip(*test_set)
    X_test = dict_vectorizer.transform(X_test)
    return X_train,X_test,y_train,y_test

In [9]:
X_train,X_test,y_train,y_test = get_train_test(train_set,test_set)
rf = RandomForestClassifier(n_estimators=100,n_jobs=4,random_state=10)
rf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=4, random_state=10)

In [10]:
preds = rf.predict(X_test)
print(accuracy_score(y_test,preds))

0.82


In [11]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
all_words_in_reviews = nltk.FreqDist(word.lower() for word in movie_reviews.words() if word not in stopwords_list)
top_words_in_reviews = [list(words) for words in zip(*all_words_in_reviews.most_common(2000))][0]

In [12]:
featuresets = [(ext_ft(d,top_words_in_reviews), c) for (d,c) in reviews]
train_set, test_set = featuresets[200:], featuresets[:200]
X_train,X_test,y_train,y_test = get_train_test(train_set,test_set)

In [13]:
rf = RandomForestClassifier(n_estimators=100,n_jobs=4,random_state=10)
rf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=4, random_state=10)

In [14]:
preds = rf.predict(X_test)
print(accuracy_score(y_test,preds))

0.845


In [15]:
features_list = zip(dict_vectorizer.get_feature_names(),rf.feature_importances_)
features_list = sorted(features_list, key=lambda x: x[1], reverse=True)
print(features_list[0:20])    

[('word_present(bad)', 0.015278811116485404), ('word_present(mess)', 0.005997853409149859), ('word_present(waste)', 0.005950727735707335), ('word_present(stupid)', 0.005908389966152629), ('word_present(boring)', 0.005483829177678856), ('word_present(worst)', 0.0053223703467062625), ('word_present(awful)', 0.004867456205234977), ('word_present(memorable)', 0.004462035825874473), ('word_present(supposed)', 0.004210139751775817), ('word_present(ridiculous)', 0.0041114459548825776), ('word_present(plot)', 0.0041025138398298674), ('word_present(outstanding)', 0.0038684196503591946), ('word_present(least)', 0.00372593495612393), ('word_present(great)', 0.0036659288901981554), ('word_present(lame)', 0.003517451172509382), ('word_present(others)', 0.0034569298134889746), ('word_present(dull)', 0.0034495907825416177), ('word_present(wasted)', 0.003385974716146842), ('word_present(effective)', 0.003143340754626261), ('word_present(unfortunately)', 0.0031249848267250924)]


C:\Users\manue\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
